In [1]:
#import requiered libraries
import pandas            as pd                       # data science essentials
import matplotlib.pyplot as plt                      # data visualization
import seaborn           as sns                      # enhanced data viz
import numpy             as np
from sklearn.model_selection import train_test_split # train-test split
from sklearn.linear_model import LogisticRegression  # logistic regression
import statsmodels.formula.api as smf                # logistic regression
from sklearn.metrics import confusion_matrix         # confusion matrix
from sklearn.metrics import roc_auc_score            # auc score
from sklearn.neighbors import KNeighborsClassifier   # KNN for classification
from sklearn.neighbors import KNeighborsRegressor    # KNN for regression
from sklearn.preprocessing import StandardScaler     # standard scaler
from sklearn.tree import DecisionTreeClassifier      # classification trees
from sklearn.tree import plot_tree                   # tree plots
import gender_guesser.detector as gender # guess gender based on (given) name


#PAth of the excel file I'm going to use for the project
file = './GOT_character_predictions.xlsx'

#Read the file for the project

GOT = pd.read_excel(io=file)

(GOT.head(n=5))

print (GOT.shape)
print(GOT.columns)
(GOT.head(n=5))

# relabeling columns
GOT.columns = ['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1','book2', 'book3','book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive']


# checking results
print (GOT.columns)
(GOT.head(n=5))

(1946, 25)
Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1_A_Game_Of_Thrones',
       'book2_A_Clash_Of_Kings', 'book3_A_Storm_Of_Swords',
       'book4_A_Feast_For_Crows', 'book5_A_Dance_with_Dragons',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive'],
      dtype='object')
Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive'],
      dtype='object')


,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,popularity,isAlive
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,NaN,NaN,...,1.0,0.0,0.0,NaN,0,0,NaN,11,0.605351,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,NaN,NaN,NaN,House Frey,Perra Royce,...,NaN,NaN,NaN,1.0,1,1,97.0,1,0.896321,1
2,3,Addison Hill,Ser,NaN,NaN,NaN,NaN,NaN,House Swyft,NaN,...,NaN,NaN,NaN,NaN,0,1,NaN,0,0.267559,1
3,4,Aemma Arryn,Queen,NaN,82.0,NaN,NaN,NaN,House Arryn,Viserys I Targaryen,...,NaN,NaN,NaN,0.0,1,1,23.0,0,0.183946,0
4,5,Sylva Santagar,Greenstone,Dornish,276.0,NaN,NaN,NaN,House Santagar,Eldon Estermont,...,NaN,NaN,NaN,1.0,1,1,29.0,0,0.043478,1


Categorical features: 'name', 'title', 'culture', 'mother', 'father', 'heir', 'house', 'spouse' Continuous features: 'dateOfBirth', 'age', 'numDeadRelations', 'popularity' Dummies features: 'male', 'book1', 'book2', 'book3', 'book4', 'book5', 'isAliveMother', 'isAliveFather', 'isAlive Heir', 'isAliveSpouse', 'isMarried', 'isNoble', 'isAlive' (response feature)

Some features names are to complex I will change it to an easier one

In [2]:
# checking each feature for missing values
GOT.isnull().sum(axis = 0)

S.No                   0
name                   0
title               1008
culture             1269
dateOfBirth         1513
mother              1925
father              1920
heir                1923
house                427
spouse              1670
book1                  0
book2                  0
book3                  0
book4                  0
book5                  0
isAliveMother       1925
isAliveFather       1920
isAliveHeir         1923
isAliveSpouse       1670
isMarried              0
isNoble                0
age                 1513
numDeadRelations       0
popularity             0
isAlive                0
dtype: int64

In [3]:
# user-defined functions

#########################
# mv_flagger
#########################
def mv_flagger(df):
    """
Flags all columns that have missing values with 'm-COLUMN_NAME'.

PARAMETERS
----------
df : DataFrame to flag missing values


RETURNS
-------
DataFrame with missing value flags."""


    for col in df:

        if df[col].isnull().astype(int).sum() > 0:
            df['m_'+col] = df[col].isnull().astype(int)
            
    return df


In the following columns, I will replace the missing values with unknown.
Will use the unknown values for feature engineering

In [4]:
# Replace missing values of spose, mother, father and heir with unknown 
GOT['spouse'] = GOT['spouse'].replace(np.nan, 'unknown')
GOT['mother'] = GOT['mother'].replace(np.nan, 'unknown')
GOT['father'] = GOT['father'].replace(np.nan, 'unknown')
GOT['heir'] = GOT['heir'].replace(np.nan, 'unknown')
GOT['house'] = GOT['house'].replace(np.nan, 'unknown')

In [5]:
# checking each feature for missing values
print(GOT.isnull().sum(axis = 0))
GOT.columns

S.No                   0
name                   0
title               1008
culture             1269
dateOfBirth         1513
mother                 0
father                 0
heir                   0
house                  0
spouse                 0
book1                  0
book2                  0
book3                  0
book4                  0
book5                  0
isAliveMother       1925
isAliveFather       1920
isAliveHeir         1923
isAliveSpouse       1670
isMarried              0
isNoble                0
age                 1513
numDeadRelations       0
popularity             0
isAlive                0
dtype: int64


Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive'],
      dtype='object')

In [6]:
# running the mv_flagger function
GOT = mv_flagger(df = GOT)

# relabeling columns
GOT.columns = ['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age']


In [7]:
#feature engineering for ages
print(GOT['age'].describe())
print("I can notice that there are certain characters who has negative age. This must be a mistake")
print('-.-.-.-.-.-.'*5)
#check the name and index position of those characters that has negative ages
print(GOT["name"][GOT["age"] < 0])
print(GOT['age'][GOT['age'] < 0])
print("There are two people who has negative ages, These are mistake and I will replace them with 0 ")
print('-.-.-.-.-.-.'*5)

#replace those negative ages with 0
GOT.loc[1684, "age"] = 0
GOT.loc[1868, "age"] = 0

#
print(GOT['age'].describe())
print('-.-.-.-.-.-.'*5)
print("There are not any negative ages now")

#replace missing values of age with the mean value
GOT['age'].fillna(GOT["age"].mean(), inplace=True)


count       433.000000
mean      -1293.563510
std       19564.340993
min     -298001.000000
25%          18.000000
50%          27.000000
75%          50.000000
max         100.000000
Name: age, dtype: float64
I can notice that there are certain characters who has negative age. This must be a mistake
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
1684    Doreah
1868    Rhaego
Name: name, dtype: object
1684   -277980.0
1868   -298001.0
Name: age, dtype: float64
There are two people who has negative ages, These are mistake and I will replace them with 0 
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
count    433.000000
mean      36.646651
std       25.876440
min        0.000000
25%       18.000000
50%       27.000000
75%       50.000000
max      100.000000
Name: age, dtype: float64
-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.
There are not any negative ages now


end of the age feature engineering

In [8]:
# STEP 1: splitting name

# placeholder list
placeholder_lst = []

# looping over each email address
for index, col in GOT.iterrows():
    
    # splitting email domain at '@'
    split_name = GOT.loc[index, 'name'].split(sep = " ")
    
    # appending placeholder_lst with the results
    placeholder_lst.append(split_name)
    

# converting placeholder_lst into a DataFrame 
name_df = pd.DataFrame(placeholder_lst)


# displaying the results
name_df

,0,1,2,3,4,5
0,Viserys,II,Targaryen,None,None,None
1,Walder,Frey,None,None,None,None
2,Addison,Hill,None,None,None,None
3,Aemma,Arryn,None,None,None,None
4,Sylva,Santagar,None,None,None,None
...,...,...,...,...,...,...
1941,Luwin,None,None,None,None,None
1942,Reek,None,None,None,None,None
1943,Symeon,Star-Eyes,None,None,None,None
1944,Coldhands,None,None,None,None,None


In [9]:
#This iteration will be used to detect the gender and if is important or not
# STEP 2: concatenating with original DataFrame

# safety measure in case of multiple concatenations
safetyGOT = pd.read_excel(file)


# renaming column to concatenate
name_df.columns = ['name 1', 'name 2', 'name 3', 'name 4', 'name 5', 'name 6']


# concatenating personal_email_domain with friends DataFrame
GOT = pd.concat([GOT, name_df['name 1']], axis = 1)


# printing value counts of personal_email_domain
GOT.loc[: ,'name 1'].value_counts()

Jon         21
Jeyne       14
Aegon       12
Alyn        10
Walder      10
            ..
Falyse       1
Cerenna      1
Roelle       1
Jalabhar     1
Rhaella      1
Name: name 1, Length: 1442, dtype: int64

In [10]:
# guessing gender based on (given) name

# placeholder list
placeholder_lst = []


# looping to guess gender
for name in GOT['name 1']:
    guess = gender.Detector().get_gender(name)
    print(guess)
    placeholder_lst.append(guess)


# converting list into a series
GOT['gender_guess'] = pd.Series(placeholder_lst)


# checking results
GOT.loc[:,["name","gender_guess"]]

unknown
unknown
andy
unknown
female
unknown
unknown
unknown
male
male
mostly_male
mostly_male
mostly_male
mostly_male
mostly_male
mostly_male
unknown
male
unknown
unknown
male
male
female
unknown
unknown
female
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
mostly_female
unknown
unknown
unknown
unknown
unknown
female
unknown
unknown
male
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
male
unknown
unknown
unknown
unknown
unknown
unknown
female
unknown
unknown
unknown
male
male
andy
andy
unknown
andy
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
female
male
male
unknown
male
male
male
male
male
male
male
mostly_male
male
mostly_male
mostly_male
unknown
unknown
unknown
male
unknown
male
unknown
male
unknown
male
unknown
unknown
male
unknown
unknown
unknown
unknown
unknown
unknown
female
unknown
mostly_male
unknown
unknown
male


unknown
unknown
unknown
unknown
unknown
unknown
unknown
male
unknown
male
unknown
unknown
unknown
unknown
unknown
male
unknown
unknown
unknown
female
unknown
male
male
male
male
unknown
unknown
unknown
unknown
unknown
unknown
unknown
male
male
male
male
male
male
male
male
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
female
unknown
unknown
unknown
unknown
unknown
unknown
unknown
female
female
male
unknown
unknown
unknown
unknown
unknown
unknown
male
male
female
unknown
female
male
unknown
unknown
unknown
unknown
unknown
unknown
male
male
male
unknown
unknown
unknown
unknown
unknown
unknown
unknown
male
unknown
male
unknown
unknown
male
unknown
female
unknown
unknown
unknown
mostly_male
male
unknown
unknown
male
male
male
unknown
female
unknown
unknown
unknown
unknown
female
unknown
female
female
female
male
male
unknown
unknown
unknown
male
male
unknown
male
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
unknown
female
unknown

,name,gender_guess
0,Viserys II Targaryen,unknown
1,Walder Frey,unknown
2,Addison Hill,andy
3,Aemma Arryn,unknown
4,Sylva Santagar,female
...,...,...
1941,Luwin,unknown
1942,Reek,unknown
1943,Symeon Star-Eyes,male
1944,Coldhands,unknown


In [11]:
#export the gender guesser to a excel file
GOT.to_excel('./gender_guesser_GOT.xlsx')

file2= './gender_guesser_GOT.xlsx'

GOT2= pd.read_excel(io=file2)

print(GOT2['gender_guess'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )

unknown          1385
male              381
female            125
mostly_male        24
mostly_female      21
andy               10
Name: gender_guess, dtype: int64


In [12]:
#dummies for gender guess
df_GOT = pd.get_dummies(GOT2, columns = ['gender_guess'], prefix = 'is')
df_GOT.head(n=5)

,Unnamed: 0,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,...,m_isAliveHeir,m_isAliveSpouse,m_age,name 1,is_andy,is_female,is_male,is_mostly_female,is_mostly_male,is_unknown
0,0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,unknown,...,0,1,1,Viserys,0,0,0,0,0,1
1,1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,unknown,unknown,unknown,House Frey,...,1,0,0,Walder,0,0,0,0,0,1
2,2,3,Addison Hill,Ser,NaN,NaN,unknown,unknown,unknown,House Swyft,...,1,1,1,Addison,1,0,0,0,0,0
3,3,4,Aemma Arryn,Queen,NaN,82.0,unknown,unknown,unknown,House Arryn,...,1,0,0,Aemma,0,0,0,0,0,1
4,4,5,Sylva Santagar,Greenstone,Dornish,276.0,unknown,unknown,unknown,House Santagar,...,1,0,0,Sylva,0,1,0,0,0,0


There are specific honored titles that belong just to male characters such as King or Prince. and there are specific honored titles that belong just to female character such as Queen or Princess. I will use this specific honored titles to find the gender of the characters.

In [13]:
# creating a dictionary to store type of variables

type_titles = {

 # full model
 'male_titles'   : [ 'King', 'Ser', 'Castellan', 'Knight', 'Maester', 
                    'Prince', 'Khal', 'Lord', 'Brother', 'Archmaester',
                    'Septon','Seneschal', 'Goodman'],
 

 # significant variables only (set 1)
 'female_title'    : ['Queen', 'Princess', 'Lady', 'Septa' ],
    

}

#this title belong just to male characters, therefore is a male
df_GOT['male_by_king'] = (df_GOT["title"].str.lower()
                            .str.contains("king", regex=False, na=False).astype(int))
df_GOT['male_by_ser'] = (df_GOT["title"].str.lower()
                            .str.contains("ser", regex=False, na=False).astype(int))
df_GOT['male_by_castellan'] = (df_GOT["title"].str.lower()
                            .str.contains("castellan", regex=False, na=False).astype(int))
df_GOT['male_by_knight'] = (df_GOT["title"].str.lower()
                            .str.contains("knight", regex=False, na=False).astype(int))
df_GOT['male_by_maester'] = (df_GOT["title"].str.lower()
                            .str.contains("maester", regex=False, na=False).astype(int))
df_GOT['male_by_prince'] = (df_GOT["title"].str.lower()
                            .str.contains("prince", regex=False, na=False).astype(int))
df_GOT['male_by_Khal'] = (df_GOT["title"].str.lower()
                            .str.contains("khal", regex=False, na=False).astype(int))
df_GOT['male_by_lord'] = (df_GOT["title"].str.lower()
                            .str.contains("lord", regex=False, na=False).astype(int))
df_GOT['male_by_brother'] = (df_GOT["title"].str.lower()
                            .str.contains("Brother", regex=False, na=False).astype(int))
df_GOT['male_by_septon'] = (df_GOT["title"].str.lower()
                            .str.contains("septon", regex=False, na=False).astype(int))

#this title belong just to female characters, therefore is a female
df_GOT['female_by_queen'] = (df_GOT["title"].str.lower()
                             .str.contains("queen", regex=False, na=False).astype(int))
df_GOT['female_by_princess'] = (df_GOT["title"].str.lower()
                             .str.contains("princess", regex=False, na=False).astype(int))
df_GOT['female_by_lady'] = (df_GOT["title"].str.lower()
                             .str.contains("lady", regex=False, na=False).astype(int))
df_GOT['female_by_septa'] = (df_GOT["title"].str.lower()
                             .str.contains("septa", regex=False, na=False).astype(int))


df_GOT = df_GOT.drop(['Unnamed: 0'],axis=1)


df_GOT.columns

Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa'],
      dtype='object')

In [14]:

column_names = ['male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon']

#list of titles that belong to female
column_names_female =['female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa']

#adding all the male by titles
df_GOT['Total_male_by_title']= df_GOT[column_names].sum(axis=1)
#adding all the categories related to gender male
df_GOT['Total_male'] = df_GOT['is_male']+ df_GOT['is_mostly_male']+df_GOT['Total_male_by_title']

#adding all the female by titles
df_GOT['Total_female_by_title']= df_GOT[column_names_female].sum(axis=1)
#adding all the categories related to gender female
df_GOT['Total_female'] = df_GOT['is_female']+ df_GOT['is_mostly_female']+df_GOT['Total_female_by_title']


# checking results
print(df_GOT['Total_male'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )

print(df_GOT['Total_female'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )


0    1100
1     639
2     205
3       2
Name: Total_male, dtype: int64
0    1754
1     173
2      19
Name: Total_female, dtype: int64


In [15]:
#dummies for the Total male and Total female 
df_GOT2 = pd.get_dummies(df_GOT, columns = ['Total_male'], prefix = 'is')

df_GOT3 = pd.get_dummies(df_GOT2, columns = ['Total_female'], prefix = 'is')
print(df_GOT3.shape)
df_GOT3.columns 

(1946, 63)


Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa', 'Total_male_by_title',
       'Total_female_by_title', 'is_0', 'is_1', 'is_2', 'is_3', 'is_0', 'is_1',
       'is_2'],
      dtype='object')

In [16]:
# relabeling columns
df_GOT3.columns = ['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa', 'Total_male_by_title',
       'Total_female_by_title','is_male_0', 'is_male_1', 'is_male_2', 'is_male_3', 
       'is_female_0', 'is_female_1','is_female_2']



In [17]:
#summing all the columns related to male gender
df_GOT3['Total_male_final'] = ( df_GOT3['is_male_1']+ df_GOT3['is_male_2']+df_GOT3['is_male_3'])

#summing all the columns related to female gender
df_GOT3['Total_female_final'] = ( df_GOT3['is_female_1']+df_GOT3['is_female_2'])

print(df_GOT3['Total_female_final'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )

print(df_GOT3['Total_male_final'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )
df_GOT3.columns

0    1754
1     192
Name: Total_female_final, dtype: int64
0    1100
1     846
Name: Total_male_final, dtype: int64


Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa', 'Total_male_by_title',
       'Total_female_by_title', 'is_male_0', 'is_male_1', 'is_male_2',
       'is_male_3', 'is_female_0', 'is_female_1', 'is_female_2

This is the end of the gender column. So far I have 53% of the gender correctly labeled
The initial rate was 28.8% using just the gender_guesser

In [18]:
#Will divide the houses columns in 2, major houses and different
#list of major houses in the story
major_houses = ['House Stark', 'House Targaryen', 'House Lannister', 'House Greyjoy',
                     'House Tyrell', 'House Baratheon', 'House Martell', 'House Arryn', 
                     'House Tully','House Frey']
placeholder= []
for index, row in df_GOT3.iterrows():
    aux = 0
    for c,house in enumerate(major_houses):
       
    # checking for house.
        if house in str(row['house']) and aux==0:
            placeholder.append(1)
            aux = 1
            continue
    if aux == 0:
        placeholder.append(0)
majorhouse_df= pd.DataFrame(placeholder)
#relabeling column
majorhouse_df.columns=['is_major_house']
df_GOT3=pd.concat([df_GOT3,majorhouse_df],axis=1)

#check the values of major house or not
print(df_GOT3['is_major_house'].value_counts(normalize = False,     
                             sort   = True,   # sort by frequencies  
                          ascending = False).round( decimals = 2)  )
df_GOT3.head(5)


0    1500
1     446
Name: is_major_house, dtype: int64


,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,is_male_0,is_male_1,is_male_2,is_male_3,is_female_0,is_female_1,is_female_2,Total_male_final,Total_female_final,is_major_house
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,unknown,unknown,...,1,0,0,0,1,0,0,0,0,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,unknown,unknown,unknown,House Frey,Perra Royce,...,0,1,0,0,1,0,0,1,0,1
2,3,Addison Hill,Ser,NaN,NaN,unknown,unknown,unknown,House Swyft,unknown,...,0,1,0,0,1,0,0,1,0,0
3,4,Aemma Arryn,Queen,NaN,82.0,unknown,unknown,unknown,House Arryn,Viserys I Targaryen,...,1,0,0,0,0,1,0,0,1,1
4,5,Sylva Santagar,Greenstone,Dornish,276.0,unknown,unknown,unknown,House Santagar,Eldon Estermont,...,1,0,0,0,0,1,0,0,1,0


This is the end for houses feature engineering
Will analalyze between major houses and no major house

In [19]:
print(df_GOT3.shape)
df_GOT3.columns

(1946, 66)


Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa', 'Total_male_by_title',
       'Total_female_by_title', 'is_male_0', 'is_male_1', 'is_male_2',
       'is_male_3', 'is_female_0', 'is_female_1', 'is_female_2

In [20]:
# checking each feature for missing values
print(df_GOT3.isnull().sum(axis = 0))
df_GOT3.columns

S.No                     0
name                     0
title                 1008
culture               1269
dateOfBirth           1513
                      ... 
is_female_1              0
is_female_2              0
Total_male_final         0
Total_female_final       0
is_major_house           0
Length: 66, dtype: int64


Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa', 'Total_male_by_title',
       'Total_female_by_title', 'is_male_0', 'is_male_1', 'is_male_2',
       'is_male_3', 'is_female_0', 'is_female_1', 'is_female_2

In [21]:
df_GOT3 = df_GOT3.drop(['is_andy', 'is_female', 'is_male', 'is_mostly_female',
       'is_mostly_male', 'is_unknown', 'male_by_king', 'male_by_ser',
       'male_by_castellan', 'male_by_knight', 'male_by_maester',
       'male_by_prince', 'male_by_Khal', 'male_by_lord', 'male_by_brother',
       'male_by_septon', 'female_by_queen', 'female_by_princess',
       'female_by_lady', 'female_by_septa'],axis=1)

print(df_GOT3.isnull().sum(axis = 0))
df_GOT3.columns

S.No                        0
name                        0
title                    1008
culture                  1269
dateOfBirth              1513
mother                      0
father                      0
heir                        0
house                       0
spouse                      0
book1                       0
book2                       0
book3                       0
book4                       0
book5                       0
isAliveMother            1925
isAliveFather            1920
isAliveHeir              1923
isAliveSpouse            1670
isMarried                   0
isNoble                     0
age                         0
numDeadRelations            0
popularity                  0
isAlive                     0
m_title                     0
m_culture                   0
m_dateOfBirth               0
m_isAliveMother             0
m_isAliveFather             0
m_isAliveHeir               0
m_isAliveSpouse             0
m_age                       0
name 1    

Index(['S.No', 'name', 'title', 'culture', 'dateOfBirth', 'mother', 'father',
       'heir', 'house', 'spouse', 'book1', 'book2', 'book3', 'book4', 'book5',
       'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse',
       'isMarried', 'isNoble', 'age', 'numDeadRelations', 'popularity',
       'isAlive', 'm_title', 'm_culture', 'm_dateOfBirth', 'm_isAliveMother',
       'm_isAliveFather', 'm_isAliveHeir', 'm_isAliveSpouse', 'm_age',
       'name 1', 'Total_male_by_title', 'Total_female_by_title', 'is_male_0',
       'is_male_1', 'is_male_2', 'is_male_3', 'is_female_0', 'is_female_1',
       'is_female_2', 'Total_male_final', 'Total_female_final',
       'is_major_house'],
      dtype='object')

In [22]:
df_corr = df_GOT3.corr(method='pearson')

df_corr['isAlive'].sort_values(ascending = False)

isAlive                  1.000000
isAliveHeir              0.384900
book4                    0.268975
isAliveFather            0.195992
isAliveSpouse            0.174275
m_age                    0.149954
m_dateOfBirth            0.149954
m_isAliveMother          0.144563
m_isAliveFather          0.137573
m_isAliveHeir            0.132652
m_isAliveSpouse          0.050037
m_title                  0.041096
is_male_0                0.037619
m_culture                0.036647
book5                    0.032846
is_female_1              0.020753
is_male_3                0.018734
book3                    0.006693
is_male_2                0.004402
Total_female_final       0.003318
is_female_0             -0.003318
Total_male_by_title     -0.031386
Total_male_final        -0.037619
isNoble                 -0.042211
isAliveMother           -0.043033
is_male_1               -0.043867
is_female_2             -0.050005
isMarried               -0.050037
is_major_house          -0.057696
book2         

In [23]:
df_GOT3.head(5)

,S.No,name,title,culture,dateOfBirth,mother,father,heir,house,spouse,...,is_male_0,is_male_1,is_male_2,is_male_3,is_female_0,is_female_1,is_female_2,Total_male_final,Total_female_final,is_major_house
0,1,Viserys II Targaryen,NaN,NaN,NaN,Rhaenyra Targaryen,Daemon Targaryen,Aegon IV Targaryen,unknown,unknown,...,1,0,0,0,1,0,0,0,0,0
1,2,Walder Frey,Lord of the Crossing,Rivermen,208.0,unknown,unknown,unknown,House Frey,Perra Royce,...,0,1,0,0,1,0,0,1,0,1
2,3,Addison Hill,Ser,NaN,NaN,unknown,unknown,unknown,House Swyft,unknown,...,0,1,0,0,1,0,0,1,0,0
3,4,Aemma Arryn,Queen,NaN,82.0,unknown,unknown,unknown,House Arryn,Viserys I Targaryen,...,1,0,0,0,0,1,0,0,1,1
4,5,Sylva Santagar,Greenstone,Dornish,276.0,unknown,unknown,unknown,House Santagar,Eldon Estermont,...,1,0,0,0,0,1,0,0,1,0


In [24]:
# declaring explanatory variables
GOT_data= df_GOT3.drop('isAlive',axis=1)


# declaring response variable
GOT_target=df_GOT3.loc[:,'isAlive']

In [25]:
# train-test split with stratification
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data,
            GOT_target,
            test_size    = 0.1,
            random_state = 219,
            stratify     = GOT_target) # preserving balance


# merging training data for statsmodels
GOT_train = pd.concat([x_train, y_train], axis = 1)

In [26]:
print(f"""

Response Variable Proportions (Training Set)
--------------------------------------------
{y_train.value_counts(normalize = True).round(decimals = 2)}



Response Variable Proportions (Testing Set)
--------------------------------------------
{y_test.value_counts(normalize = True).round(decimals = 2)}
""")



Response Variable Proportions (Training Set)
--------------------------------------------
1    0.75
0    0.25
Name: isAlive, dtype: float64



Response Variable Proportions (Testing Set)
--------------------------------------------
1    0.74
0    0.26
Name: isAlive, dtype: float64



In [27]:
for var in GOT_train.keys():
    print(var+ ' +')

S.No +
name +
title +
culture +
dateOfBirth +
mother +
father +
heir +
house +
spouse +
book1 +
book2 +
book3 +
book4 +
book5 +
isAliveMother +
isAliveFather +
isAliveHeir +
isAliveSpouse +
isMarried +
isNoble +
age +
numDeadRelations +
popularity +
m_title +
m_culture +
m_dateOfBirth +
m_isAliveMother +
m_isAliveFather +
m_isAliveHeir +
m_isAliveSpouse +
m_age +
name 1 +
Total_male_by_title +
Total_female_by_title +
is_male_0 +
is_male_1 +
is_male_2 +
is_male_3 +
is_female_0 +
is_female_1 +
is_female_2 +
Total_male_final +
Total_female_final +
is_major_house +
isAlive +


In [28]:
# instantiating a logistic regression model object
logistic_small = smf.logit(formula   = """isAlive ~ 
numDeadRelations +
popularity+
age+
book1 +
book2 +
book4""",
                           data = GOT_train)


# FITTING the model object
results_logistic = logistic_small.fit()


# checking the results SUMMARY
results_logistic.summary2() # summary2() has AIC and BIC

Optimization terminated successfully.
         Current function value: 0.498211
         Iterations 6


<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.121     
Dependent Variable: isAlive          AIC:              1758.7362 
Date:               2021-12-05 19:09 BIC:              1797.0118 
No. Observations:   1751             Log-Likelihood:   -872.37   
Df Model:           6                LL-Null:          -992.53   
Df Residuals:       1744             LLR p-value:      4.7682e-49
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------
Intercept          1.6196   0.2047  7.9132 0.0000  1.2185  2.0208
numDeadRelations  -0.1500   0.0521 -2.8784 0.0040 -0.2522 -0.0479
popularity        -1.3853   0.4480 -3.0920 0.0020 -2.2635 -0.5072
age               -0.0214   0.0047 -4.5199 0.0000 -0.0307 -0.0121
book1             -0.4920   0.1542 -3.1899 0.0014 -0.7943 -0.1897
book2             -0.4477   0.1343 -3.3344 0.0009 -0.7109 -0.1846
book4              1.4171   0.1273 11.1319 0.0000  1.1676  1.6666
=================================================================

"""

the final features I choose to run all my models are:
numDeadRelations + popularity+ age+ book1 + book2 + book4 

In [29]:
GOT_dict = {

 # full model
 'logit_sig'   : ['numDeadRelations' , 'popularity' , 'age', 'book1', 'book2' ,'book4']}

From this point, I will start running the models

LOGIT MODEL

In [30]:
#LOGIT MODEL
# declaring explanatory variables
GOT_data= df_GOT3.loc[ : , GOT_dict['logit_sig']]


# declaring response variable
GOT_target=df_GOT3.loc[:,'isAlive']



# Split the data into train and test
x_train, x_test, y_train, y_test = train_test_split(
            GOT_data,
            GOT_target,
            test_size    = 0.1,
            random_state = 219,
            stratify     = GOT_target)


# INSTANTIATING a logistic regression model
logreg = LogisticRegression(solver = 'lbfgs',
                            C = 1,
                            random_state = 219)


# FITTING the training data
logreg_fit = logreg.fit(x_train, y_train)


# PREDICTING based on the testing set
logreg_pred = logreg_fit.predict(x_test)


# SCORING the results
print(' Logit Training ACCURACY:', logreg_fit.score(x_train, y_train).round(4))
print('Logit Testing  ACCURACY:', logreg_fit.score(x_test, y_test).round(4))


# saving scoring data for future use
logreg_train_score = logreg_fit.score(x_train, y_train).round(4) # accuracy
logreg_test_score  = logreg_fit.score(x_test, y_test).round(4) # accuracy

print(f""" Logit train-test GAP:""" , abs(logreg_train_score- logreg_test_score ).round(4))

 Logit Training ACCURACY: 0.7727
Logit Testing  ACCURACY: 0.8256
 Logit train-test GAP: 0.0529


In [31]:
# unpacking the confusion matrix
logreg_tn, \
logreg_fp, \
logreg_fn, \
logreg_tp = confusion_matrix(y_true = y_test, y_pred = logreg_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {logreg_tn}
False Positives: {logreg_fp}
False Negatives: {logreg_fn}
True Positives : {logreg_tp}
""")


True Negatives : 19
False Positives: 31
False Negatives: 3
True Positives : 142



In [32]:
# area under the roc curve (auc)
print(roc_auc_score(y_true  = y_test,
                    y_score = logreg_pred).round(decimals = 4))


# saving AUC score for future use
logreg_auc_score = roc_auc_score(y_true  = y_test,
                                 y_score = logreg_pred).round(decimals = 4)

0.6797


CART MODEL

In [33]:
# INSTANTIATING a classification tree object
full_tree = DecisionTreeClassifier()


# FITTING the training data
full_tree_fit = full_tree.fit(x_train, y_train)


# PREDICTING on new data
full_tree_pred = full_tree_fit.predict(x_test)


# SCORING the model
print('Full Tree Training ACCURACY:', full_tree_fit.score(x_train,
                                                     y_train).round(4))

print('Full Tree Testing ACCURACY :', full_tree_fit.score(x_test,
                                                     y_test).round(4))

print('Full Tree AUC Score:', roc_auc_score(y_true  = y_test,
                                            y_score = full_tree_pred).round(4))


# saving scoring data for future use
full_tree_train_score = full_tree_fit.score(x_train, y_train).round(4) # accuracy
full_tree_test_score  = full_tree_fit.score(x_test, y_test).round(4)   # accuracy


# saving AUC
full_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                      y_score = full_tree_pred).round(4) # auc

print(f""" Logit train-test GAP:""" , abs(full_tree_train_score- full_tree_test_score  ).round(4))

Full Tree Training ACCURACY: 0.8943
Full Tree Testing ACCURACY : 0.7744
Full Tree AUC Score: 0.7041
 Logit train-test GAP: 0.1199


In [34]:
# unpacking the confusion matrix
full_tree_tn, \
full_tree_fp, \
full_tree_fn, \
full_tree_tp = confusion_matrix(y_true = y_test, y_pred = full_tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {full_tree_tn}
False Positives: {full_tree_fp}
False Negatives: {full_tree_fn}
True Positives : {full_tree_tp}
""")


True Negatives : 28
False Positives: 22
False Negatives: 22
True Positives : 123



Pruned Tree Model

In [35]:
# INSTANTIATING a classification tree object
tree_pruned = DecisionTreeClassifier(max_depth = 8 ,
                              min_samples_leaf = 25,
                                  random_state = 219)


# FITTING the training data
tree_pruned_fit = tree_pruned.fit(x_train, y_train)


# PREDICTING on new data
tree_pred = tree_pruned_fit.predict(x_test)


# SCORING the model
print('Training ACCURACY:', tree_pruned_fit.score(x_train, y_train).round(4))
print('Testing  ACCURACY:', tree_pruned_fit.score(x_test, y_test).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = tree_pred).round(4))


# saving scoring data for future use
pruned_tree_train_score = tree_pruned_fit.score(x_train, y_train).round(4) # accuracy
pruned_tree_test_score  = tree_pruned_fit.score(x_test, y_test).round(4) # accuracy


# saving auc score
pruned_tree_auc_score   = roc_auc_score(y_true  = y_test,
                                        y_score = tree_pred).round(4) # auc

print(f""" Logit train-test GAP:""" , abs(pruned_tree_train_score- pruned_tree_test_score  ).round(4))

Training ACCURACY: 0.7978
Testing  ACCURACY: 0.8256
AUC Score        : 0.7255
 Logit train-test GAP: 0.0278


In [36]:
# unpacking the confusion matrix
pruned_tree_tn, \
pruned_tree_fp, \
pruned_tree_fn, \
pruned_tree_tp = confusion_matrix(y_true = y_test, y_pred = tree_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {pruned_tree_tn}
False Positives: {pruned_tree_fp}
False Negatives: {pruned_tree_fn}
True Positives : {pruned_tree_tp}
""")


True Negatives : 26
False Positives: 24
False Negatives: 10
True Positives : 135



KNN MODEL

In [41]:
########################################
# optimal_neighbors
########################################
def optimal_neighbors(x_data,
                      y_data,
                      standardize = True,
                      pct_test=0.25,
                      seed=219,
                      response_type='reg',
                      max_neighbors=20,
                      show_viz=True):
    """
Exhaustively compute training and testing results for KNN across
[1, max_neighbors]. Outputs the maximum test score and (by default) a
visualization of the results.
PARAMETERS
----------
x_data        : explanatory variable data
y_data        : response variable
standardize   : whether or not to standardize the x data, default True
pct_test      : test size for training and validation from (0,1), default 0.25
seed          : random seed to be used in algorithm, default 219
response_type : type of neighbors algorithm to use, default 'reg'
    Use 'reg' for regression (KNeighborsRegressor)
    Use 'class' for classification (KNeighborsClassifier)
max_neighbors : maximum number of neighbors in exhaustive search, default 20
show_viz      : display or surpress k-neigbors visualization, default True
"""    
    
    
    if standardize == True:
        # optionally standardizing x_data
        scaler             = StandardScaler()
        scaler.fit(x_data)
        x_scaled           = scaler.transform(x_data)
        x_scaled_df        = pd.DataFrame(x_scaled)
        x_data             = x_scaled_df



    # train-test split
    x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                        y_data,
                                                        test_size = pct_test,
                                                        random_state = seed)


    # creating lists for training set accuracy and test set accuracy
    training_accuracy = []
    test_accuracy = []
    
    
    # setting neighbor range
    neighbors_settings = range(1, max_neighbors + 1)


    for n_neighbors in neighbors_settings:
        # building the model based on response variable type
        if response_type == 'reg':
            clf = KNeighborsRegressor(n_neighbors = n_neighbors)
            clf.fit(x_train, y_train)
            
        elif response_type == 'class':
            clf = KNeighborsClassifier(n_neighbors = n_neighbors)
            clf.fit(x_train, y_train)            
            
        else:
            print("Error: response_type must be 'reg' or 'class'")
        
        
        # recording the training set accuracy
        training_accuracy.append(clf.score(x_train, y_train))
    
        # recording the generalization accuracy
        test_accuracy.append(clf.score(x_test, y_test))


    # optionally displaying visualization
    if show_viz == True:
        # plotting the visualization
        fig, ax = plt.subplots(figsize=(12,8))
        plt.plot(neighbors_settings, training_accuracy, label = "training accuracy")
        plt.plot(neighbors_settings, test_accuracy, label = "test accuracy")
        plt.ylabel("Accuracy")
        plt.xlabel("n_neighbors")
        plt.legend()
        plt.show()
    
    
    # returning optimal number of neighbors
    print(f"The optimal number of neighbors is: {test_accuracy.index(max(test_accuracy))+1}")
    return test_accuracy.index(max(test_accuracy))+1


########################################
# visual_cm
########################################
def visual_cm(true_y, pred_y, labels = None):
    """
Creates a visualization of a confusion matrix.

PARAMETERS
----------
true_y : true values for the response variable
pred_y : predicted values for the response variable
labels : , default None
    """
    # visualizing the confusion matrix

    # setting labels
    lbls = labels
    

    # declaring a confusion matrix object
    cm = confusion_matrix(y_true = true_y,
                          y_pred = pred_y)


    # heatmap
    sns.heatmap(cm,
                annot       = True,
                xticklabels = lbls,
                yticklabels = lbls,
                cmap        = 'Blues',
                fmt         = 'g')


    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix of the Classifier')
    plt.show()

In [38]:
# INSTANTIATING StandardScaler()
scaler = StandardScaler()


# FITTING the data
scaler.fit(GOT_data)


# TRANSFORMING the data
x_scaled     = scaler.transform(GOT_data)


# converting to a DataFrame
x_scaled_df  = pd.DataFrame(x_scaled) 


# train-test split with the scaled data
x_train_scaled, x_test_scaled, y_train_scaled, y_test_scaled = train_test_split(
            x_scaled_df,
            GOT_target,
            random_state = 219,
            test_size    = 0.1,
            stratify     = GOT_target)


# INSTANTIATING a KNN classification model with optimal neighbors
knn_opt = KNeighborsClassifier(n_neighbors = 10)


# FITTING the training data
knn_fit = knn_opt.fit(x_train_scaled, y_train_scaled)


# PREDICTING based on the testing set
knn_pred = knn_fit.predict(x_test_scaled)


# SCORING the results
print('Training ACCURACY:', knn_fit.score(x_train_scaled, y_train_scaled).round(4))
print('Testing  ACCURACY:', knn_fit.score(x_test_scaled, y_test_scaled).round(4))
print('AUC Score        :', roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4))


# saving scoring data
knn_train_score = knn_fit.score(x_train_scaled, y_train_scaled).round(4)
knn_test_score  = knn_fit.score(x_test_scaled, y_test_scaled).round(4)
print(f""" Logit train-test GAP:""" , abs(knn_train_score- knn_test_score  ).round(4))

# saving AUC score
knn_auc_score   = roc_auc_score(y_true  = y_test,
                                          y_score = knn_pred).round(4)


Training ACCURACY: 0.8144
Testing  ACCURACY: 0.8462
AUC Score        : 0.7786
 Logit train-test GAP: 0.0318


In [39]:
# unpacking the confusion matrix
knn_tree_tn, \
knn_tree_fp, \
knn_tree_fn, \
knn_tree_tp = confusion_matrix(y_true = y_test, y_pred = knn_pred).ravel()


# printing each result one-by-one
print(f"""
True Negatives : {knn_tree_tn}
False Positives: {knn_tree_fp}
False Negatives: {knn_tree_fn}
True Positives : {knn_tree_tp}
""")


True Negatives : 32
False Positives: 18
False Negatives: 12
True Positives : 133



In [40]:
# comparing results
print(f"""
Model         AUC Score       TN, FP, FN, TP
-----         ---------       --------------
Logistic      {logreg_auc_score}         {logreg_tn, logreg_fp, logreg_fn, logreg_tp}
Full Tree     {full_tree_auc_score}         {full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp}
Pruned Tree   {pruned_tree_auc_score}         {pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp}
knn (I choose){knn_auc_score}         {knn_tree_tn,knn_tree_fp,knn_tree_fn,knn_tree_tp}
""")


# creating a dictionary for model results
model_performance = {
    
    'Model Name'    : ['Logistic', 'Full Tree', 'Pruned Tree', 'Knn'],
           
    'AUC Score' : [logreg_auc_score, full_tree_auc_score, pruned_tree_auc_score,knn_auc_score],
    
    'Training Accuracy' : [logreg_train_score, full_tree_train_score,
                           pruned_tree_train_score,knn_train_score],
           
    'Testing Accuracy'  : [logreg_test_score, full_tree_test_score,
                           pruned_tree_test_score,knn_test_score],

    'Confusion Matrix'  : [(logreg_tn, logreg_fp, logreg_fn, logreg_tp),
                           (full_tree_tn, full_tree_fp, full_tree_fn, full_tree_tp),
                           (pruned_tree_tn, pruned_tree_fp, pruned_tree_fn, pruned_tree_tp),
                          (knn_tree_tn, knn_tree_fp, knn_tree_fn, knn_tree_tp)]}




Model         AUC Score       TN, FP, FN, TP
-----         ---------       --------------
Logistic      0.6797         (19, 31, 3, 142)
Full Tree     0.7041         (28, 22, 22, 123)
Pruned Tree   0.7255         (26, 24, 10, 135)
knn (I choose)0.7786         (32, 18, 12, 133)

